In [1]:
""" 
Fine tune a image captioning or description model on a artwork description data to
obtain a model that is able to analyze an artwork and generate a description of 
the artwork that is hopefully more art-review-like than a generic image captioning 
model would

The pretrained model used here is the Salesforce Blip Captioniong model: 
https://huggingface.co/Salesforce/blip-image-captioning-base

The fine tuning data set is the data from the SemArt Project: 
https://github.com/noagarcia/SemArt
"""

' \nFine tune a image captioning or description model on a artwork description data to\nobtain a model that is able to analyze an artwork and generate a description of \nthe artwork that is hopefully more art-review-like than a generic image captioning \nmodel would\n\nThe pretrained model used here is the Salesforce Blip Captioniong model: \nhttps://huggingface.co/Salesforce/blip-image-captioning-base\n\nThe fine tuning data set is the data from the SemArt Project: \nhttps://github.com/noagarcia/SemArt\n'

In [2]:
import os
import torch
import pandas as pd
import chardet
from PIL import Image
from torchvision import transforms
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import datasets
from datasets import Dataset as HFDataset, DatasetDict, IterableDataset
from transformers import BlipProcessor, BlipForConditionalGeneration, ViTImageProcessor, VisionEncoderDecoderModel, AutoTokenizer, TrainingArguments, Trainer

In [3]:
def list_files_in_directory(dir_path):
    try:
        files = os.listdir(dir_path)
        return [x for x in files if '.jpg' in x]
    except FileNotFoundError:
        return f"Directory not found: {dir_path}"
    except NotADirectoryError:
        return f"Not a directory: {dir_path}"

In [4]:
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
import os

# Step 1: Load the CSV file as a dictionary mapping image names to descriptions
semart_dir = f"/Users/rckyi/Documents/Datasette/SemArtData/SemArt" 
images_dir = semart_dir 

description_file_train = semart_dir + '/semart_train.csv'
description_file_test = semart_dir + '/semart_test.csv'

with open(description_file_train, 'rb') as file:
    print(f'file path {description_file_train}')
    result = chardet.detect(file.read())
    encoding = result['encoding']
    print(f'encoding {encoding}')
    df_train = pd.read_csv(description_file_train, encoding=encoding, sep='\t')
    print(type(df_train['IMAGE_FILE'][0]))
# df_test = pd.read_csv(description_file_test, encoding = "utf-8")


image_to_description = dict(zip(df_train["IMAGE_FILE"], df_train["DESCRIPTION"]))
# list(image_to_description.items())[0:5]

# # Define the image directory and transformation
# image_dir = "path/to/images"
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a fixed shape
    transforms.ToTensor(),  # Convert image to tensor
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])


file path /Users/rckyi/Documents/Datasette/SemArtData/SemArt/semart_train.csv
encoding utf-8
<class 'str'>


In [5]:
# # Step 1: Load the CSV file into a dictionary mapping image names to descriptions
# description_file = "path/to/description.csv"
# df = pd.read_csv(description_file)
image_files_in_dir = list_files_in_directory(semart_dir+'/Images/')
# image_files_in_dir
# list(image_to_description.items())[0]

In [6]:
from datasets import IterableDataset, DatasetDict
from transformers import BlipProcessor, BlipForConditionalGeneration, TrainingArguments, Trainer
import torch

def data_generator(images_name_list):
    ret = []
    image_file_list = list(df_train['IMAGE_FILE'])
#     def generator():
    for image_name in images_name_list:
#         image_path = os.path.join(images_dir, image_name)
        image_path = f"{semart_dir}/Images/{image_name}"

        if image_name in image_file_list: #image_to_description: # and os.path.exists(image_path):
            image = Image.open(image_path).convert("RGB")
            image_tensor = transform(image)
#             ret.append({"image":image_tensor, "text":image_to_description[image_name]})
            yield {"image":image_tensor, "text":image_to_description[image_name]}
#     return ret

In [7]:
data_var = data_generator(image_files_in_dir)

In [8]:
# print(data_var)

In [9]:
hf_dataset = DatasetDict({"train": datasets.Dataset.from_list(data_var)})
hf_dataset

TypeError: 'generator' object is not subscriptable

In [ ]:
def transforms(example_batch):
#     ((V + 1.0) * 255/2).astype(np.uint8)
    images = np.asarray([x for x in example_batch["image"]])
    captions = [x for x in example_batch["text"]]
    inputs = processor(images=images, text=captions, padding="max_length")
    inputs.update({"labels": inputs["input_ids"]})
    return inputs

hf_dataset.set_transform(transforms)

In [ ]:
hf_dataset

In [ ]:
# Define a custom collator to handle the format expected by the BLIP model
def collate_fn(batch):
#     print(f'batch {batch}')
    images = [item['image'] for item in batch]
    texts = [item['text'] for item in batch]
    # Convert the images into tensor and normalize
    images = torch.stack(images)
    images = torch.add(images,1.0)
    text = torch.stack(texts)
    return processor(images=images, text=texts, padding=True, return_tensors="pt")


In [ ]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./blip-finetuned",
    per_device_train_batch_size=8,
    num_train_epochs=5,
    save_steps=500,
    evaluation_strategy=None,
#     evaluation_strategy="steps",
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    push_to_hub=False,  # Set to True if uploading to Hugging Face Hub
    logging_steps=10,
    fp16=torch.cuda.is_available(),
    remove_unused_columns=False,
    max_steps=10,
    no_cuda=True,
)

In [ ]:
# Load Pretrained BLIP Model & Processor
model_name = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# # Define Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=hf_dataset["train"].with_format("torch"),
#     tokenizer=processor,
# )

In [ ]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= hf_dataset["train"],
    tokenizer=processor,
#     data_collator=collate_fn,  # Use custom collate function to prepare batches
)

In [ ]:
# hf_dataset["train"]

In [ ]:
# Start Fine-Tuning
trainer.train()
